<div style="text-align: right">Peter Norvig<br>Mar 2024</div> 

# Stubborn Number Endings

[Francis Su](https://www.francissu.com/)'s book *Mathematics for Human Flourishing* mentions the fact that numbers that end in "5" have a square that also ends in "5". 

For example, 5² = 25, 15² = 225, and 25² = 625. 

This leads to some questions:

- Is there an easy way to calculate the square of a number ending in "5"?
- What should we call this property of "square has same ending"?
- Are there other digits besides 5 that have this  property?
- Can we prove the property, not just show some examples?
- Are there multi-digit endings that have this property?


## Is there an easy way to calculate the square of a number ending in "5"?

Let's make a table of {number: square} pairs and try to see a pattern:

In [1]:
{i: i ** 2 for i in range(15, 100, 10)}

{15: 225,
 25: 625,
 35: 1225,
 45: 2025,
 55: 3025,
 65: 4225,
 75: 5625,
 85: 7225,
 95: 9025}

I see a pattern here: for example 95 squared is 9025, which you get by taking the "9", multiplying it by one more than itself to get  9⋅10 = 90, then squaring 5 to get 25, then putting the "90" next to the "25" to get "9025".

Does the trick also work for numbers with more digits? Let's investigate:

In [2]:
{i: i ** 2 for i in range(105, 250, 10)}

{105: 11025,
 115: 13225,
 125: 15625,
 135: 18225,
 145: 21025,
 155: 24025,
 165: 27225,
 175: 30625,
 185: 34225,
 195: 38025,
 205: 42025,
 215: 46225,
 225: 50625,
 235: 55225,
 245: 60025}

Yes, it looks like the trick still works: 105 squared is 10⋅11 = "110", followed by "25," to make "11025". And 245 squared is 24⋅25 = "600", followed by "25", to make "60025". 


## What should we call this property?

Let's define an **ending** as the rightmost-digits (zero or more) of a number in decimal notation. 

Then we can say that an ending is **stubborn**  if every number with that ending has a square with the same ending.

## Are there other digits besides 5 that are stubborn?

We could work this out in our heads, or with paper and pencil, or we can compute it with an expression:

In [15]:
digits = '0123456789'

{e for e in digits if str(int(e) ** 2).endswith(e)}

{'0', '1', '5', '6'}

These are the four digits whose square ends in the same digit.

## Can we prove stubborness?

We have seen that 0² ends in 0, 1² ends in 1, 5² ends in 5, and 6² ends in 6. And we have checked some numbers with those endings, for example, 245² ends in 5. But can we **prove** that **every** number ending in 0, 1, 5, or 6 has a square that ends in the same digit?

Some notation: I'll use quote marks, as in "*se*" to mean the string of staring digits "*s*" followed by the string of ending digits "*e*".

With a little bit of algebra we can see that if *s* is any string of digits and *e* is a single ending digit,  then:

"*se*"² = (10⋅*s* + *e*)² = (10⋅*s*)² + 2⋅(10⋅*s* ⋅ *e*) + *e*² = 10 ⋅ (10⋅*s*² + 2⋅*s*⋅*e*) + *e*²

This is ten times some integer, plus *e*², so "*se*"² ends in the digit *e* if and only if *e*² ends in *e*, and we know that is true for 0, 1, 5, and 6, and for no other digits.

## Are there multi-digit endings that are stubborn?

The algebraic argument above can be extended to work with an ending string *e* that is *k* digits long:

"*se*"² = (10<sup>*k*</sup>⋅*s* + *e*)² = (10<sup>*k*</sup>⋅*s*)² + 2⋅(10<sup>*k*</sup>⋅*s* ⋅ *e*) + *e*² = 10<sup>*k*</sup> ⋅ (10<sup>*k*</sup>⋅*s*² + 2⋅*s*⋅*e*) + *e*²

This is 10<sup>*k*</sup> times some integer, plus *e*², so again "*se*"² ends in *e* if and only if *e*² ends in *e*. To put it another way, to test whether *e* is stubborn, all we have to do is square *e* and see if the result ends in *e*. There is one complication: we'd like to say that "00" is stubborn, because any number ending in "00", when squared, also ends in "00", for example 200² = 40000. But 00² is zero, which we write as "0", not as "00" or "0000". To make sure that "00" is considered stubborn, I will define the predicate function `stubborn(ending)` to test the square of "1" followed by the ending (I could have chosen any other starting digit string and the result would be the same). 


In [4]:
def stubborn(ending: str) -> bool:
    """Does the square of any number with this ending also end with this ending?"""
    return str(int("1" + ending) ** 2).endswith(ending)

Now we can find all two-digit stubborn endings as follows:

In [17]:
[s + e for s in digits for e in digits if stubborn(s + e)]

['00', '01', '25', '76']

We could easily continue on in this way, enumerating all three-, four- or even six-digit endings, and checking each one to see if it is stubborn.  There are only a million six-digit endings. But there are a quadrillion 15-digit endings, so it would take a *long* time to check all of those. And 100-digit endings? Forget about it.  Instead we need to rely on a simplification:
- Any two-digit stubborn ending ('00', '01', '25', '76') has to end in a one-digit-stubborn ending ('0', '1', '5', '6').
- In general, any *d*-digit stubborn ending has to end in a (*d*-1)-digit stubborn ending.
- So, to find the stubborn endings of length 100, I don't need to generate and test all 10<sup>100</sup> endings, I only need to consider the stubborn endings of length 99 and check each one of them. (If this simplification is not obvious, stop and convince yourself it is true.)

Using this simplification we can efficiently compute all stubborn-endings of a given length *d* as follows (caching greatly improves efficiency):

In [6]:
from functools import lru_cache

@lru_cache(None)
def stubborn_endings(d: int) -> list:
    """A list of all stubborn endings of length `d` digits."""
    if d == 0:
        return [''] # The empty ending is the sole stubborn ending of length 0.
    else:
        return [(s + e) for e in stubborn_endings(d - 1) 
                        for s in digits 
                        if stubborn(s + e)]

For example:

In [7]:
stubborn_endings(3)

['000', '001', '625', '376']

In [8]:
stubborn_endings(4)

['0000', '0001', '0625', '9376']

In [9]:
stubborn_endings(5)

['00000', '00001', '90625', '09376']

In [10]:
stubborn_endings(6)

['000000', '000001', '890625', '109376']

In [11]:
stubborn_endings(100)

['0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001',
 '3953007319108169802938509890062166509580863811000557423423230896109004106619977392256259918212890625',
 '6046992680891830197061490109937833490419136188999442576576769103890995893380022607743740081787109376']

## More questions!

This leads to a few new questions.

## Can each stubborn ending be extended exactly one way?

We know that each stubborn ending of length *d* has to build on the endings of length *d - 1*, but is it always the case that for any length *d* there will be exactly four endings? Might there be a case where two digits work with one of the endings, or no digits?

I can show that there are always 4 endings up to length *d* = 2000, but I leave it to you to describe a proof that this will always be true.

In [12]:
for d in range(1, 2000):
    assert len(stubborn_endings(d)) == 4 

One cool implication: if it is true that a stubborn ending of any length can always be extended, that means there is an infinitely long integer whose square ends with itself!

## What digits are used to extend each ending?

There doesn't seem to be a pattern, and all digits seemingly get used roughly evenly. 

I don't have a theory of which digit comes next, but I can count how many times each digit appears in the 2000-digit endings that end in "5" and "6", and see that each of the ten digits appears about 200 times:

In [13]:
from collections import Counter

zero, one, five, six = stubborn_endings(2000)

Counter(five)

Counter({'0': 205,
         '3': 173,
         '2': 208,
         '6': 197,
         '9': 198,
         '5': 197,
         '4': 206,
         '8': 214,
         '7': 205,
         '1': 197})

In [14]:
Counter(six)

Counter({'9': 205,
         '6': 174,
         '7': 208,
         '3': 197,
         '0': 198,
         '4': 196,
         '5': 206,
         '1': 214,
         '2': 205,
         '8': 197})